In [1]:
import pandas as pd
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import numpy as np
import json
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge geocoder --yes
import geocoder
import geopy
import requests

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

This section is responsible for read the csv file and creating a data frame from it

In [2]:
file_name = 'https://www.doogal.co.uk/AdministrativeAreasCSV.ashx?district=E09000023' 
df = pd.read_csv(file_name)
df.head()

,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,Ward,Parish,Introduced,Terminated,Altitude,Country,Last Updated,Quality,LSOA Code
0,BR1 4BY,Yes,51.417289,-0.001741,539050,170591,TQ390705,Downham,"Lewisham, unparished area",1980-01-01,NaN,35,England,2019-05-29,Within the building of the matched address clo...,E01003235
1,BR1 4DN,Yes,51.418996,-0.002156,539016,170780,TQ390707,Downham,"Lewisham, unparished area",1980-01-01,NaN,35,England,2019-05-29,Within the building of the matched address clo...,E01003235
2,BR1 4EY,Yes,51.418477,0.005042,539518,170736,TQ395707,Downham,"Lewisham, unparished area",1980-01-01,NaN,50,England,2019-05-29,Within the building of the matched address clo...,E01032565
3,BR1 4FD,Yes,51.421083,-0.002194,539007,171012,TQ390710,Downham,"Lewisham, unparished area",2010-01-01,NaN,33,England,2019-05-29,Within the building of the matched address clo...,E01003235
4,BR1 4JG,Yes,51.419403,-0.000728,539114,170828,TQ391708,Downham,"Lewisham, unparished area",1980-01-01,NaN,40,England,2019-05-29,Within the building of the matched address clo...,E01003235


We start Data Cleaning process here

In [3]:
df_clean = df[['Postcode','Latitude','Longitude','Ward','Altitude']]
print('There are {} wards in the Lewisham Council.'.format(len(df_clean['Ward'].unique())))
df_clean.head()

There are 18 wards in the Lewisham Council.


,Postcode,Latitude,Longitude,Ward,Altitude
0,BR1 4BY,51.417289,-0.001741,Downham,35
1,BR1 4DN,51.418996,-0.002156,Downham,35
2,BR1 4EY,51.418477,0.005042,Downham,50
3,BR1 4FD,51.421083,-0.002194,Downham,33
4,BR1 4JG,51.419403,-0.000728,Downham,40


In [4]:
df_LeeGreen = df_clean[df_clean['Ward'] == 'Lee Green']
print("Number of Postalcodes in LeeGraan area: ", len(df_LeeGreen))
df_LeeGreen = df_LeeGreen.reset_index(drop=True)
df_LeeGreen.head()

Number of Postalcodes in LeeGraan area:  296


,Postcode,Latitude,Longitude,Ward,Altitude
0,SE12 0AA,51.449266,0.013068,Lee Green,24
1,SE12 0AB,51.449620,0.012923,Lee Green,23
2,SE12 0AD,51.449781,0.013983,Lee Green,24
3,SE12 0AE,51.447524,0.013956,Lee Green,30
4,SE12 0AF,51.446609,0.014851,Lee Green,35


In [5]:
def Assign_sector(row):
    row['Sector'] = row.Postcode.split(' ',1)[1]
    return row
df_LeeGreen = df_LeeGreen.apply(Assign_sector,axis = 1)
df_LeeGreen.head()

,Postcode,Latitude,Longitude,Ward,Altitude,Sector
0,SE12 0AA,51.449266,0.013068,Lee Green,24,0AA
1,SE12 0AB,51.449620,0.012923,Lee Green,23,0AB
2,SE12 0AD,51.449781,0.013983,Lee Green,24,0AD
3,SE12 0AE,51.447524,0.013956,Lee Green,30,0AE
4,SE12 0AF,51.446609,0.014851,Lee Green,35,0AF


This section is responsible for locating the are, LeeGreen on the map using Folium

In [6]:
address = 'Lee Green, London'
geolocator = Nominatim(user_agent = 'chrome') 
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lee Green, London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lee Green, London are 51.3913435, 0.1021205.


In [7]:
map_Lewisham = folium.Map(location = [latitude, longitude], zoom_start=12)
for lat, long, ward, sector in zip(df_LeeGreen['Latitude'],df_LeeGreen['Longitude'],df_LeeGreen['Ward'],df_LeeGreen['Sector']):
    label = '{}, {}'.format(ward,sector)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat,long],
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color='white',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Lewisham)
map_Lewisham

In [8]:
CLIENT_ID = 'WZMKEEYWU10QTC3ODEXLFN0Q0PCVJMV4CLFS53H1QNDSZX3M'
CLIENT_SECRET = '30A1WGVYSO5JAV4YWMUAFVCTANRPY4WCKQ1RKYTWXYUIUSS0'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WZMKEEYWU10QTC3ODEXLFN0Q0PCVJMV4CLFS53H1QNDSZX3M
CLIENT_SECRET:30A1WGVYSO5JAV4YWMUAFVCTANRPY4WCKQ1RKYTWXYUIUSS0


In [9]:
LIMIT = 10
radius = 50
def getNearbyVenues(names, latitudes, longitudes):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}' \
            .format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']        
        # return only relevant information for each nearby venue
        venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name'])
                            for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood','Neighbourhood Latitude','Neighbourhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']    
    return(nearby_venues)

In [10]:
LeeGreen_venues = getNearbyVenues(names=df_LeeGreen['Sector'],latitudes=df_LeeGreen['Latitude'],longitudes=df_LeeGreen['Longitude'])
print("Number of venues in LeeGreen: ", len(LeeGreen_venues))
print('There are {} uniques categories.'.format(len(LeeGreen_venues['Venue Category'].unique())))
LeeGreen_venues.head()

0AA
0AB
0AD
0AE
0AF
0AG
0HL
0HN
0HP
0HR
0HS
0JE
0JF
0JH
0JJ
0JL
0JN
0JW
0JY
0RD
0RE
0RN
0RP
0RR
0RW
0SA
0SB
0SJ
0SL
0SN
0SP
0SR
0SS
0ST
0SU
0SW
0SX
0SY
0SZ
0TA
0TB
0TD
0TE
0TF
0TG
0TH
0TJ
0TL
0TN
0TQ
0TT
0TU
0TX
0TZ
0UA
0UF
8AA
8AB
8AD
8AE
8AG
8AH
8AJ
8AQ
8AT
8AU
8AX
8AY
8AZ
8BB
8BF
8BG
8DD
8DJ
8DL
8DN
8DP
8DR
8DS
8DT
8DU
8DW
8DX
8DY
8DZ
8EA
8EB
8ED
8EF
8EH
8LG
8LH
8LJ
8LL
8LN
8LP
8LQ
8LR
8LS
8LT
8LU
8LX
8LY
8LZ
8NA
8NB
8NN
8NP
8NR
8NS
8NT
8NU
8NW
8NX
8NY
8NZ
8PA
8PB
8PD
8PE
8PF
8PG
8PH
8PJ
8PL
8PN
8PQ
8PS
8PU
8PW
8PY
8PZ
8QA
8QB
8QD
8QE
8QF
8QG
8QH
8QJ
8QL
8QN
8QP
8QQ
8QW
8RA
8RB
8RE
8RF
8RG
8RH
8RN
8RR
8RS
8RT
8RW
8RX
8RY
8SR
8SS
8ST
8SU
8SX
8SY
8SZ
8TA
8TD
8TE
8TF
8TG
8TH
8TJ
8TL
8TN
8TP
8TQ
8TR
8ZA
8ZB
8ZD
8ZE
8ZF
8ZG
8ZH
8ZL
8ZQ
8ZU
8ZZ
9HP
9HR
9HS
9HT
9HU
9HX
9HY
9HZ
9JA
9JB
9JD
9JG
9JH
9JQ
9JW
5EZ
5FB
5FD
5FE
5FG
5HA
5HB
5HF
5HG
5HH
5HJ
5HL
5HN
5HP
5HQ
5HR
5HS
5HU
5HW
5HX
5HY
5HZ
5JA
5JB
5JD
5JE
5LP
5LR
5LS
5LT
5LU
5LW
5LX
5LY
5LZ
5NA
5NB
5ND
5NE
5NF
5NG
5NH
5NJ
5NL
5NN
5NP
5NQ


In [16]:
LeeGreen_venues.rename(columns={'Neighbourhood':'Sector'}, inplace=True)
LeeGreen_venues.head()

,Sector,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0AA,51.449266,0.013068,Lee Food & Wine,51.449270,0.013170,Convenience Store
1,0AA,51.449266,0.013068,Lee Station Bus Stop B,51.449493,0.012869,Bus Stop
2,0AA,51.449266,0.013068,The Kitchen SE12,51.448926,0.013130,Café
3,0AB,51.449620,0.012923,Lee Railway Station (LEE),51.449706,0.013239,Train Station
4,0AB,51.449620,0.012923,Lee Food & Wine,51.449270,0.013170,Convenience Store


In [17]:
LeeGreen_onehot = pd.get_dummies(LeeGreen_venues[['Venue Category']],  prefix="", prefix_sep="")
LeeGreen_onehot['Sector'] = LeeGreen_venues['Sector']
fixed_columns = [LeeGreen_onehot.columns[-1]] + list(LeeGreen_onehot.columns[:-1])
LeeGreen_onehot = LeeGreen_onehot[fixed_columns]
LeeGreen_grouped = LeeGreen_onehot.groupby('Sector').mean().reset_index()
LeeGreen_grouped.shape

(47, 26)

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Sector']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
Sector_venues_sorted = pd.DataFrame(columns=columns)
Sector_venues_sorted['Sector'] = LeeGreen_grouped['Sector']
for ind in np.arange(LeeGreen_grouped.shape[0]):
    Sector_venues_sorted.iloc[ind, 1:] = return_most_common_venues(LeeGreen_grouped.iloc[ind, :], num_top_venues)
Sector_venues_sorted[0:10]

,Sector,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0AA,Bus Stop,Café,Convenience Store,Train Station,Gastropub,Chinese Restaurant,Chocolate Shop,Coffee Shop,Electronics Store,Fish & Chips Shop
1,0AB,Train Station,Bus Stop,Convenience Store,Gastropub,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Electronics Store,Fish & Chips Shop
2,0AE,Bus Stop,Fried Chicken Joint,Train Station,Gastropub,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store
3,0JE,Bus Stop,Train Station,Gastropub,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,Fish & Chips Shop
4,0JF,Bus Stop,Convenience Store,Train Station,Gastropub,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Electronics Store,Fish & Chips Shop
5,0JW,Bus Stop,Train Station,Gastropub,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,Fish & Chips Shop
6,0SX,Grocery Store,Café,Fried Chicken Joint,Gastropub,Bus Stop,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store
7,5FB,Asian Restaurant,Convenience Store,Gastropub,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Electronics Store,Fish & Chips Shop
8,5FD,Café,Convenience Store,Fish & Chips Shop,Flower Shop,Fried Chicken Joint,Train Station,Gastropub,Bus Stop,Chinese Restaurant,Chocolate Shop
9,5FE,Rental Car Location,Train Station,Gastropub,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store


K-means Clustering

In [20]:
kclusters = 5
LeeGreen_grouped_clustering = LeeGreen_grouped.drop('Sector', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(LeeGreen_grouped_clustering)
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 1, 3, 3, 4, 3, 3, 2, 2, 3, 3, 4, 0, 3, 3, 3, 0, 0,
       2, 3, 3, 3, 3, 2, 2, 2, 3, 2, 3, 3, 2, 3, 1, 2, 3, 3, 3, 3, 3, 1, 1,
       1], dtype=int32)

In [21]:
LeeGreen_merged = Sector_venues_sorted
LeeGreen_merged['Cluster Labels'] = kmeans.labels_
LeeGreen_merged2 = pd.merge(df_LeeGreen,LeeGreen_merged, on='Sector')
LeeGreen_merged2.head()

,Postcode,Latitude,Longitude,Ward,Altitude,Sector,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,SE12 0AA,51.449266,0.013068,Lee Green,24,0AA,Bus Stop,Café,Convenience Store,Train Station,Gastropub,Chinese Restaurant,Chocolate Shop,Coffee Shop,Electronics Store,Fish & Chips Shop,2
1,SE12 0AB,51.449620,0.012923,Lee Green,23,0AB,Train Station,Bus Stop,Convenience Store,Gastropub,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Electronics Store,Fish & Chips Shop,2
2,SE12 0AE,51.447524,0.013956,Lee Green,30,0AE,Bus Stop,Fried Chicken Joint,Train Station,Gastropub,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,2
3,SE12 0JE,51.449710,0.012340,Lee Green,22,0JE,Bus Stop,Train Station,Gastropub,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,Fish & Chips Shop,2
4,SE12 0JF,51.449213,0.012505,Lee Green,23,0JF,Bus Stop,Convenience Store,Train Station,Gastropub,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Electronics Store,Fish & Chips Shop,2


In [23]:
# Create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(LeeGreen_merged2['Latitude'], LeeGreen_merged2['Longitude'], LeeGreen_merged2['Sector'], LeeGreen_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)       
map_clusters

Cluster 1

In [24]:
LeeGreen_merged.loc[LeeGreen_merged['Cluster Labels'] == 0, LeeGreen_merged.columns[[1] + list(range(5, LeeGreen_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
17,Pub,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,0
21,Pub,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,0
22,Pub,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,0


Cluster 2

In [25]:
LeeGreen_merged.loc[LeeGreen_merged['Cluster Labels'] == 1, LeeGreen_merged.columns[[1] + list(range(5, LeeGreen_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
6,Grocery Store,Bus Stop,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,1
37,Grocery Store,Café,Chinese Restaurant,Chocolate Shop,Convenience Store,Electronics Store,Fish & Chips Shop,1
44,Grocery Store,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,Fish & Chips Shop,1
45,Grocery Store,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,1
46,Grocery Store,Bus Stop,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,1


Cluster 3

In [26]:
LeeGreen_merged.loc[LeeGreen_merged['Cluster Labels'] == 2, LeeGreen_merged.columns[[1] + list(range(5, LeeGreen_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Bus Stop,Gastropub,Chinese Restaurant,Chocolate Shop,Coffee Shop,Electronics Store,Fish & Chips Shop,2
1,Train Station,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Electronics Store,Fish & Chips Shop,2
2,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,2
3,Bus Stop,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,Fish & Chips Shop,2
4,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Electronics Store,Fish & Chips Shop,2
5,Bus Stop,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,Fish & Chips Shop,2
12,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Electronics Store,Fish & Chips Shop,2
13,Bus Stop,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,Fish & Chips Shop,2
23,Bus Stop,Gastropub,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,2
28,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,2


Cluster 4

In [27]:
LeeGreen_merged.loc[LeeGreen_merged['Cluster Labels'] == 3, LeeGreen_merged.columns[[1] + list(range(5, LeeGreen_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
7,Asian Restaurant,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Electronics Store,Fish & Chips Shop,3
8,Café,Fried Chicken Joint,Train Station,Gastropub,Bus Stop,Chinese Restaurant,Chocolate Shop,3
10,Gastropub,Convenience Store,Fish & Chips Shop,Flower Shop,Fried Chicken Joint,Train Station,Bus Stop,3
11,Café,Convenience Store,Fish & Chips Shop,Flower Shop,Fried Chicken Joint,Train Station,Bus Stop,3
14,Asian Restaurant,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Electronics Store,3
15,Electronics Store,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Fish & Chips Shop,3
18,Hostel,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,3
19,Fish & Chips Shop,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,3
20,Gym / Fitness Center,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,3
24,Pharmacy,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Electronics Store,3


Cluster 5

In [28]:
LeeGreen_merged.loc[LeeGreen_merged['Cluster Labels'] == 4, LeeGreen_merged.columns[[1] + list(range(5, LeeGreen_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
9,Rental Car Location,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,4
16,Rental Car Location,Café,Chinese Restaurant,Chocolate Shop,Coffee Shop,Convenience Store,Electronics Store,4
